# annotation

> This module contains code for the annotation tab of the application

In [ ]:
#| default_exp selection

In [ ]:
#| hide
from nbdev.showdoc import *

These are the relevant library calls to build out the UI and the calls to the model in order to set up the manuscript selection buttons.

In [3]:
#| hide
# This library is only for testing the layout. It does not need to be called extraneously
from dash import Dash

In [2]:
#| export
from dash import dcc, html
import plotly.express as px
import cv2

## createAnnotationTextArea

This creates a ```Texarea``` object. Without changing some of the JavaScript code, the ```Textarea``` object does not have the ability to display line numbers.

In [ ]:
#| export
def createAnnotationTextArea():
    return dcc.Textarea(id="annotation",value="Enter transcription text here!",style={"width":"100%","height":285})

In [ ]:
#| hide
app = Dash(__name__)

app.layout = html.Div(
    [
        createAnnotationTextArea()
    ]
)

if __name__ == "__main__":
    app.run(debug=True)

## createPageSelector

For any manuscript that is in use, it is not unexpected for there to be multiple pages. In the case that there are multiple pages, it would be good for users to be able to select them. To that end, this function creates a [```Dropdown```](https://dash.plotly.com/dash-core-components/dropdown) object. By the nature of this application, the ```options``` will have to be dynamically assigned during run time.

In [ ]:
#| export
def createPageSelector():
    return dcc.Dropdown(id="page-selector")

In [ ]:
#| hide
test = createPageSelector()
test.options = ['a','b','c']

test

## createAnnotationFigure

The heart of this program is right here. This is how boxes will be drawn. In order to do this, we will use the [```Imshow```](https://plotly.com/python/imshow/) chart from the ```Plotly``` library. There are [ways](https://dash.plotly.com/annotations) of linking this figure into the application. Fortunately or unfortunately, it is not possible to keep one plot object with an image inside it. Instead, the figure needs to be made again and again for each image fed into it. Then, this plot is wrapped in the ```Graph``` object from the ```Dash``` library to be given an ```id``` for callback features. This wrapping needs to be handled in the ```Dash``` app itself, not the view calls.

This function takes in a pathway to a manuscript image as a string and returns a ```Plotly``` figure.

Due to some quirks of the ```cv2``` library, using ```cv2.imread()``` will return an image with BGR ordering of color channels, not RGB ordering of color channels. Computer monitors use the RGB color scheme as a way of representing colors for display:

<img
     style = "display:block;
              margin-left:auto;
              margin-right: auto;
              width:20%;"
     src = "https://upload.wikimedia.org/wikipedia/commons/9/97/RGB_color_wheel_pixel_30.svg"
/>

In an LED monitor, every pixel is made of a red, blue, and green LED that will show different intensities of colored light. Combining these pixels together into one system allows us to display images, movies, videos, etc... by coordingating the display of colors for each LED in each pixel of a monitor. What does this have to do with images? In mathematical terms, we might say that an image $I$ will have scalar intensities associated to three different coordinates. The first are the traditional $(x,y)$ Cartesian coordinate structure, but the third coordinate is color. We might say then that an individual LED at some position on the monitor will display the value $I(x_0,y_0,c)$ at the LED (where $c$ is theoretically the associated color channel). Most monitors use RGB (where red is the first index, green is the second index, and green is the third index), but the ```cv2``` library uses the BGR scheme. It is essentially the same as the RGB scheme, but a computer monitor will display red as if it is blue and blue as if it is red, distorting the image. To fix this, this function automatically reverses the ordering of BGR to RGB, hence the slicing.

In [4]:
#| export
def createAnnotationFigure(path):
    img = cv2.imread(path)
    # This reorders the color channels (the first two indices relate to the intensity values of individual colors while the last index indicates what
    img = img[:,:,::-1]
    fig = px.imshow(img)
    fig.update_layout(dragmode='drawrect',
                  # style of new shapes
                  newshape=dict(line_color='grey',
                                opacity=0.6))
    return fig

In [ ]:
#| hide
fig = createAnnotationFigure('glyptodon/manuscripts/stvrnktmnstrygrkcllctnn.53/images/15_01_0053_0006_f_3r_res.png')
fig.show(config={'modeBarButtonsToAdd':['drawline','drawrect','eraseshape']})

In [6]:
#| hide
pathOptions = [
    "glyptodon/manuscripts/stvrnktmnstrygrkcllctnn.53/images/15_01_0053_0011_f_5v_res.png",
    "glyptodon/manuscripts/stvrnktmnstrygrkcllctnn.53/images/15_01_0053_0006_f_3r_res.png",
]
config = {"modeBarButtonsToAdd": ["drawline", "drawrect", "eraseshape"]}

app = Dash(__name__)
app.layout = html.Div(
    [
        dcc.Graph(
            id="test-image",
            figure=createAnnotationFigure(pathOptions[1]),
            config=config,
            style={
                "height": 900,
                "width": 800,
            },
        )
    ]
)

if __name__ == "__main__":
    app.run()

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()